In [2]:
import threading

In [6]:
ThreadLocalVar = threading.local()
ThreadLocalVar.a = 5
ThreadLocalVar.b = 10
print(a, b)

NameError: name 'a' is not defined

In [7]:
ThreadLocalVar

In [ ]:
def func_a(message="", times=10):
    for count in times: 
        print(message)
        print(datetime.now().strftime("%Y-%m-%d %H:%M:%s"))

In [ ]:
x = threading.Thread(func_a)

In [ ]:
x.start()

In [ ]:
x.run()